In [ ]:
!pip install "snowflake-connector-python[pandas]"

In [3]:
import os
from datetime import datetime
from snowflake.snowpark import Session
from snowflake.connector.pandas_tools import write_pandas
import pandas as pd
import re
import json


In [37]:
#withdraw data#
def create_snowflake_session(secret_file_path):
    with open(secret_file_path, 'r') as file:
        connection_parameters = json.load(file)
    session = Session.builder.configs(connection_parameters).create()
    return session

def execute_sql(session, sql_query):
    result = session.sql(sql_query).collect()
    df = pd.DataFrame([row.as_dict() for row in result])
    return df

def upload_to_snowflake(session, df, table_name):


    # Ensure all columns are properly formatted
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].astype(str)
        elif pd.api.types.is_numeric_dtype(df[col]):
            df[col] = pd.to_numeric(df[col], errors='coerce')
    result = session.write_pandas( 
                                df = df, 
                                table_name = table_name, 
                                schema='TRUSTED',
                                overwrite=True,
                                auto_create_table = True,
                                table_type = 'transient'
                                )
    return result

secret_file_path = '../.secret'
session = create_snowflake_session(secret_file_path)
get_raw_data_sql = 'select * from train'
df_train_raw = execute_sql(session = session, sql_query= get_raw_data_sql)
df_train_raw.to_csv('../data/train_raw.csv')

In [27]:
# preprocess_helper.py
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

def load_data_dictionary(file_path):
    data_dictionary = pd.read_csv(file_path)
    data_dictionary.columns = data_dictionary.columns.str.lower()
    data_dictionary['col_name'] = data_dictionary['columns_cleaned'].str.lower()
    return data_dictionary

def clean_numeric_columns(df, data_dictionary):
    numeric_columns = data_dictionary[data_dictionary['data_type'] == 'NUMBER']['col_name'].tolist()
    for col in numeric_columns:
        if col in df.columns:
            if df[col].dtype == 'object':
                df[col] = df[col].str.replace(',', '').str.replace('%', '').str.replace('$', '').astype(float, errors='ignore')
        else:
            print(f'Column {col} not found in the training data')
    return df

def apply_fill_method(df, data_dictionary):
    print("Applying fill methods...")
    for _, row in data_dictionary.iterrows():
        col_name = row['col_name']
        fill_method = row['fill_method']
        
        if fill_method == 'constant':
            if col_name in df.columns:
                fill_value = row['fill_value'] if pd.notna(row['fill_value']) else 0
                print(f"Filling missing values in {col_name} with {fill_value}")
                df[col_name] = df[col_name].fillna(fill_value)
            else:
                print(f"Warning: Column {col_name} not found for fill method 'constant'")
        elif fill_method == 'previous':
            prev_col_name = col_name.replace('yr1', 'yr0')
            if col_name in df.columns and prev_col_name in df.columns:
                print(f"Filling missing values in {col_name} with values from {prev_col_name}")
                df[col_name] = df[col_name].fillna(df[prev_col_name])
            else:
                print(f"Warning: Column {col_name} or {prev_col_name} not found for fill method 'previous'")
        # Add more fill methods as needed
    return df

def date_string_to_day(x, level):
    x = pd.to_datetime(x, errors='coerce')
    if level == 'month':
        return x.dt.day
    elif level == 'year':
        return x.dt.dayofyear
    return x

def subset_train_data(df, data_dictionary):
    # Select columns for training
    columns_for_training = data_dictionary[data_dictionary['use_for_training'] == 'Y']['col_name'].tolist()
    train_data_subset = df[columns_for_training]
    return train_data_subset

# config
data_dictionary_path = '../data/fundtap-data-dictionary.csv'
train_data_path = '../data/train_raw.csv'

# Load data dictionary
data_dict = load_data_dictionary(data_dictionary_path)

# Load training data
train_data = pd.read_csv(train_data_path,index_col= 0)
train_data.columns = train_data.columns.str.lower()

# Clean numeric columns
train_data = clean_numeric_columns(train_data, data_dict)

# Apply fill methods for missing values
train_data = apply_fill_method(train_data, data_dict)

# Subset data for training
train_data_subset = subset_train_data(train_data, data_dict)


/var/folders/wh/0n9ffgc50rlgjmhfj0m9s62h0000gn/T/ipykernel_32585/2313452486.py:69: DtypeWarning: Columns (7,14,31,38,39,40,41,42,43,44,45,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,87,88,89,90,91,92,93,96,97,98,121,126,144,149,162) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv(train_data_path,index_col= 0)


Applying fill methods...
Filling missing values in consumer_credit_judgements_guar1 with 0
Filling missing values in consumer_credit_insolvency_notices_guar1 with 0
Filling missing values in consumer_credit_credit_defaults_guar1 with 0
Filling missing values in consumer_credit_company_affiliations_guar1 with 0
Filling missing values in consumer_credit_file_activity_guar1 with 0
Filling missing values in consumer_credit_score_guar1 with 0
Filling missing values in consumer_credit_risk_odds_guar1 with 0
Filling missing values in consumer_credit_judgements_guar2 with 0
Filling missing values in consumer_credit_insolvency_notices_guar2 with 0
Filling missing values in consumer_credit_credit_defaults_guar2 with 0
Filling missing values in consumer_credit_company_affiliations_guar2 with 0
Filling missing values in consumer_credit_file_activity_guar2 with 0
Filling missing values in consumer_credit_score_guar2 with 0
Filling missing values in consumer_credit_risk_odds_guar2 with 0
Filling mis

In [31]:
train_data.describe()


,consumer_credit_judgements_guar2,consumer_credit_insolvency_notices_guar2,consumer_credit_credit_defaults_guar2,consumer_credit_company_affiliations_guar2,consumer_credit_file_activity_guar2,consumer_credit_risk_odds_guar2,consumer_credit_judgements_guar3,consumer_credit_insolvency_notices_guar3,consumer_credit_credit_defaults_guar3,consumer_credit_company_affiliations_guar3,...,total_non_current_assets,total_assets,accounts_payable,total_current_liabilities,total_non_current_liabilities,total_liabilities,net_assets,current_year_earnings,retained_earnings,total_equity
count,8579.0,8579.000000,8579.000000,8579.000000,8579.000000,8579.000000,8579.0,8579.0,8579.000000,8579.000000,...,8.579000e+03,8.579000e+03,8.579000e+03,8.579000e+03,8.579000e+03,8.579000e+03,8.579000e+03,8.579000e+03,8.579000e+03,8.579000e+03
mean,0.0,0.000117,0.019932,2.574309,2.855228,7.379004,0.0,0.0,0.009791,2.681898,...,2.626951e+04,2.268832e+05,6.020024e+04,1.221165e+05,5.559873e+04,1.784039e+05,4.847924e+04,2.177204e+04,1.420211e+04,4.851734e+04
std,0.0,0.010796,0.232474,5.634588,3.886688,13.007320,0.0,0.0,0.098471,4.052219,...,1.353531e+05,5.123545e+05,1.221346e+05,2.299248e+05,2.919159e+05,4.220136e+05,4.064331e+05,1.728739e+05,3.280732e+05,4.064439e+05
min,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,-1.894000e+05,-1.044593e+06,-3.439490e+04,-1.679908e+06,-7.577945e+05,-5.623589e+05,-5.049773e+06,-2.591858e+06,-7.902253e+06,-5.049773e+06
25%,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-3.977878e+04,0.000000e+00,-1.223000e+04,-3.977878e+04
50%,0.0,0.000000,0.000000,2.000000,1.000000,0.050000,0.0,0.0,0.000000,0.000000,...,0.000000e+00,9.217639e+04,1.248847e+04,4.381222e+04,0.000000e+00,7.021154e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.0,0.000000,0.000000,2.000000,4.000000,4.798000,0.0,0.0,0.000000,9.000000,...,0.000000e+00,2.591465e+05,8.592732e+04,2.099810e+05,5.544440e+04,2.286605e+05,5.675418e+04,4.549376e+04,2.029640e+03,5.675418e+04
max,0.0,1.000000,4.000000,109.000000,40.000000,73.517000,0.0,0.0,1.000000,9.000000,...,2.286581e+06,6.924974e+06,4.024932e+06,5.374071e+06,4.113556e+06,7.873054e+06,5.759333e+06,1.765624e+06,2.394022e+06,5.759333e+06


In [36]:

# # Upload full cleaned data to Snowflake
upload_to_snowflake(session, train_data, 'TRAIN_DATA_CLEANED')

# # Upload subset of training columns to Snowflake
upload_to_snowflake(session, train_data_subset, 'TRAIN_DATA_SUBSET')


/Users/jiah/huiyu/FundTapMLOps/.venv/lib/python3.11/site-packages/snowflake/snowpark/session.py:2367: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.base.Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  success, _, _, ci_output = write_pandas(
/var/folders/wh/0n9ffgc50rlgjmhfj0m9s62h0000gn/T/ipykernel_32585/2878156213.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(str)
/var/folders/wh/0n9ffgc50rlgjmhfj0m9s62h0000gn/T/ipykernel_32585/2878156213.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [38]:
train_downloaded = execute_sql(session = session, sql_query= 'SELECT * FROM FUNDTAP.TRUSTED.TRAIN_DATA_SUBSET')
train_downloaded.to_csv('../data/train.csv')

In [ ]:

# def apply_normalization(df, data_dictionary):
#     print("Applying normalization...")
#     for _, row in data_dictionary.iterrows():
#         col_name = row['col_name']
#         date_col = row['normalization_date_column']
#         level = row['normalization_level']
#         if pd.notna(date_col):
#             if col_name in df.columns and date_col in df.columns:
#                 print(f"Normalizing {col_name} based on {date_col} at {level} level")
#                 try:
#                     df['day'] = df[date_col].apply(lambda x: date_string_to_day(x, level))
#                     df[col_name] = df[col_name] / df['day']
#                 except Exception as e:
#                     print(f"Error normalizing {col_name} based on {date_col}: {e}")
#             else:
#                 print(f"Warning: Column {col_name} or {date_col} not found for normalization")
#     df.drop(columns=['day'], errors='ignore', inplace=True)
#     return df
